In [1]:
# Assignment 1

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('/content/Kmeans_assignment_data.csv')

In [4]:
df

,Name,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F_Undergrad,p_Undergrad,Outstate,Room_Board,Books,Personal,PhD,Terminal,S_F_Ratio,perc_alumni,Expend,Grad_Rate
0,Abilene Christian University,Yes,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60
1,Adelphi University,Yes,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56
2,Adrian College,Yes,1428,1097,336,22,50,1036,99,11250,3750,400,1165,53,66,12.9,30,8735,54
3,Agnes Scott College,Yes,417,349,137,60,89,510,63,12960,5450,450,875,92,97,7.7,37,19016,59
4,Alaska Pacific University,Yes,193,146,55,16,44,249,869,7560,4120,800,1500,76,72,11.9,2,10922,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
772,Worcester State College,No,2197,1515,543,4,26,3089,2029,6797,3900,500,1200,60,60,21.0,14,4469,40
773,Xavier University,Yes,1959,1805,695,24,47,2849,1107,11520,4960,600,1250,73,75,13.3,31,9189,83
774,Xavier University of Louisiana,Yes,2097,1915,695,34,61,2793,166,6900,4200,617,781,67,75,14.4,20,8323,49
775,Yale University,Yes,10705,2453,1317,95,99,5217,83,19840,6510,630,2115,96,96,5.8,49,40386,99


In [5]:
df.columns

Index(['Name', 'Private', 'Apps', 'Accept', 'Enroll', 'Top10perc', 'Top25perc',
       'F_Undergrad', 'p_Undergrad', 'Outstate', 'Room_Board', 'Books',
       'Personal', 'PhD', 'Terminal', 'S_F_Ratio', 'perc_alumni', 'Expend',
       'Grad_Rate'],
      dtype='object')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 777 entries, 0 to 776
Data columns (total 19 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         777 non-null    object 
 1   Private      777 non-null    object 
 2   Apps         777 non-null    int64  
 3   Accept       777 non-null    int64  
 4   Enroll       777 non-null    int64  
 5   Top10perc    777 non-null    int64  
 6   Top25perc    777 non-null    int64  
 7   F_Undergrad  777 non-null    int64  
 8   p_Undergrad  777 non-null    int64  
 9   Outstate     777 non-null    int64  
 10  Room_Board   777 non-null    int64  
 11  Books        777 non-null    int64  
 12  Personal     777 non-null    int64  
 13  PhD          777 non-null    int64  
 14  Terminal     777 non-null    int64  
 15  S_F_Ratio    777 non-null    float64
 16  perc_alumni  777 non-null    int64  
 17  Expend       777 non-null    int64  
 18  Grad_Rate    777 non-null    int64  
dtypes: float

In [23]:
# Define a dictionary to map old column names to new column names
column_mapping = {
    'Roll No': 'roll_no',
    'Name': 'name',
    'Quiz1+Quiz2': 'quiz_total',
    'Ass1+Ass2': 'assignment_total',
    'Mid': 'mid_term',
    'Terminal': 'terminal_exam',
    'Obtained': 'obtained_marks'
}
# Rename columns in the dataset
df.rename(columns=column_mapping, inplace=True)



print("Column names updated and changes saved.")

Column names updated and changes saved.


In [24]:
df.columns

Index(['roll_no', 'name', 'quiz_total', 'assignment_total', 'mid_term',
       'terminal_exam', 'obtained_marks'],
      dtype='object')

In [44]:
df

,roll_no,name,quiz_total,assignment_total,mid_term,terminal_exam,obtained_marks
0,1.0,Ashir Mehfooz,13.0,13.0,41.0,41.0,108.0
1,2.0,Atif Raftad,4.0,4.0,30.0,30.0,68.0
2,3.0,Saiqa Aziz,14.0,14.0,34.0,34.0,96.0
3,8.0,Ozair Minhas,4.0,4.0,12.0,12.0,32.0
4,9.0,Naveera Subhani,4.0,4.0,34.0,34.0,76.0
...,...,...,...,...,...,...,...
210,217.0,Aslan,13.0,13.0,34.0,34.0,94.0
211,218.0,Yelda,12.0,12.0,24.0,24.0,72.0
212,219.0,Ertan,15.0,15.0,11.0,11.0,52.0
213,220.0,Cansın,13.0,13.0,21.0,21.0,68.0


In [13]:
import sqlite3
import pandas as pd

# Function to create a connection to the SQLite database
def create_connection(db_file):
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print(e)
    return None

# Function to create tables in the database
def create_tables(conn):
    cursor = conn.cursor()

    # Create Marks table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS marks (
            roll_no INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            quiz_total FLOAT,
            assignment_total FLOAT,
            mid_term FLOAT,
            terminal_exam FLOAT,
            obtained_marks FLOAT
        )
    ''')
    conn.commit()

# Function to load data from dataset into the Marks table
def load_marks_data(conn, dataset_file):
    cursor = conn.cursor()

    # Read dataset
    dataset = pd.read_excel('/content/Univeristy_Results.xls')

    # Check the column names
    print(dataset.columns)
    # Or you can print the first few rows to inspect the data
    print(dataset.head())

    # Extract relevant columns for marks table
    marks_data = dataset[['Roll No', 'Name', 'Quiz1+Quiz2', 'Ass1+Ass2', 'Mid', 'Terminal', 'Obtained']]

    conn.commit()


# Function to load sample data into the Patient_Medications table
def load_sample_patient_medications(conn):
    cursor = conn.cursor()

    # Sample data for Patient_Medications table
    cursor.execute("INSERT INTO Patient_Medications (patient_id, medication_id) VALUES (?, ?)",
                   (1, 1))  # Assigning Medication A to Patient 1
    cursor.execute("INSERT INTO Patient_Medications (patient_id, medication_id) VALUES (?, ?)",
                   (2, 2))  # Assigning Medication B to Patient 2

    conn.commit()

def main():
    database = "data_warehouse.db"
    dataset_file = "/content/Univeristy_Results.xls"  #   dataset

    # Create a database connection
    conn = create_connection(database)
    if conn is not None:
        # Create tables
        create_tables(conn)
        # Load data from dataset into Patients table
        load_marks_data(conn, dataset_file)
        print("Data warehouse created successfully.")
        conn.close()
    else:
        print("Error! Cannot create the database connection.")

if __name__ == '__main__':
    main()

Index(['Roll No', 'Name', 'Quiz1+Quiz2', 'Ass1+Ass2', 'Mid', 'Terminal',
       'Obtained'],
      dtype='object')
  Roll No             Name  Quiz1+Quiz2  Ass1+Ass2 Mid Terminal  Obtained
0       1    Ashir Mehfooz         13.0       13.0  41       41     108.0
1       2      Atif Raftad          4.0        4.0  30       30      68.0
2       3       Saiqa Aziz         14.0       14.0  34       34      96.0
3       8     Ozair Minhas          4.0        4.0  12       12      32.0
4       9  Naveera Subhani          4.0        4.0  34       34      76.0
Data warehouse created successfully.


In [39]:
import sqlite3

# Function to create a connection to the SQLite database
def create_connection(db_file):
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print(e)
    return None

# Function to create tables in the database
def create_tables(conn):
    cursor = conn.cursor()

    # Create Assignment table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Assignment (
            id INTEGER PRIMARY KEY,
            roll_no TEXT NOT NULL,
            name TEXT NOT NULL,
            ass1 REAL,
            ass2 REAL,
            UNIQUE(roll_no, name)
        )
    ''')

    # Create Quiz table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Quiz (
            id INTEGER PRIMARY KEY,
            roll_no TEXT NOT NULL,
            name TEXT NOT NULL,
            quiz1 REAL,
            quiz2 REAL,
            UNIQUE(roll_no, name)
        )
    ''')

    # Create Total_Marks table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Total_Marks (
            id INTEGER PRIMARY KEY,
            roll_no TEXT NOT NULL,
            name TEXT NOT NULL,
            mid REAL,
            terminal REAL,
            UNIQUE(roll_no, name)
        )
    ''')

    # Create Student_Record table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Student_Record (
            id INTEGER PRIMARY KEY,
            roll_no TEXT NOT NULL,
            name TEXT NOT NULL,
            obtained REAL,
            UNIQUE(roll_no, name)
        )
    ''')

    # Create marks table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS marks (
            roll_no INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            quiz_total FLOAT,
            assignment_total FLOAT,
            mid_term FLOAT,
            terminal_exam FLOAT,
            obtained_marks FLOAT
        )
    ''')

    conn.commit()

def main():
    database = "data_warehouse.db"

    # Create a database connection
    conn = create_connection(database)
    if conn is not None:
        # Create tables
        create_tables(conn)
        print("Tables created successfully.")
        conn.close()
    else:
        print("Error! Cannot create the database connection.")

if __name__ == '__main__':
    main()


Tables created successfully.


In [40]:
import sqlite3

# Function to create a connection to the SQLite database
def create_connection(db_file):
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print(e)
    return None

# Function to display tables and their schemas
def display_tables_and_schema(conn):
    cursor = conn.cursor()

    # List all tables in the database
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    print("Tables in the database:")
    for table in tables:
        table_name = table[0]
        print(f"\nSchema of the {table_name} table:")
        cursor.execute(f"PRAGMA table_info('{table_name}')")
        schema = cursor.fetchall()
        for column in schema:
            print(column)

def main():
    database = "data_warehouse.db"

    # Create a database connection
    conn = create_connection(database)
    if conn is not None:
        # Display tables and their schemas
        display_tables_and_schema(conn)
        conn.close()
    else:
        print("Error! Cannot create the database connection.")

if __name__ == '__main__':
    main()


Tables in the database:

Schema of the Assignment table:
(0, 'id', 'INTEGER', 0, None, 1)
(1, 'roll_no', 'TEXT', 1, None, 0)
(2, 'name', 'TEXT', 1, None, 0)
(3, 'ass1', 'REAL', 0, None, 0)
(4, 'ass2', 'REAL', 0, None, 0)

Schema of the Quiz table:
(0, 'id', 'INTEGER', 0, None, 1)
(1, 'roll_no', 'TEXT', 1, None, 0)
(2, 'name', 'TEXT', 1, None, 0)
(3, 'quiz1', 'REAL', 0, None, 0)
(4, 'quiz2', 'REAL', 0, None, 0)

Schema of the Total_Marks table:
(0, 'id', 'INTEGER', 0, None, 1)
(1, 'roll_no', 'TEXT', 1, None, 0)
(2, 'name', 'TEXT', 1, None, 0)
(3, 'mid', 'REAL', 0, None, 0)
(4, 'terminal', 'REAL', 0, None, 0)

Schema of the Student_Record table:
(0, 'id', 'INTEGER', 0, None, 1)
(1, 'roll_no', 'TEXT', 1, None, 0)
(2, 'name', 'TEXT', 1, None, 0)
(3, 'obtained', 'REAL', 0, None, 0)

Schema of the marks table:
(0, 'roll_no', 'INTEGER', 0, None, 1)
(1, 'name', 'TEXT', 1, None, 0)
(2, 'quiz_total', 'FLOAT', 0, None, 0)
(3, 'assignment_total', 'FLOAT', 0, None, 0)
(4, 'mid_term', 'FLOAT', 0, No

In [41]:
import sqlite3
import pandas as pd
from tabulate import tabulate

# Function to create a connection to the SQLite database
def create_connection(db_file):
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print(e)
    return None

def create_fact_table(conn):
    cursor = conn.cursor()

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Marks_Fact (
            roll_no TEXT,
            name TEXT,
            quiz_total FLOAT,
            assignment_total FLOAT,
            mid_term FLOAT,
            terminal_exam FLOAT,
            obtained_marks FLOAT,
            PRIMARY KEY (roll_no, name)
        )
    ''')

    conn.commit()

def populate_fact_table(conn, dataset_file):
    cursor = conn.cursor()

    # Read dataset
    marks_data = pd.read_excel(dataset_file)

    # Insert data into Marks_Fact table
    marks_data.to_sql('Marks_Fact', conn, if_exists='replace', index=False)

    conn.commit()

def display_fact_table(conn):
    cursor = conn.cursor()

    cursor.execute('''
        SELECT *
        FROM Marks_Fact
    ''')

    rows = cursor.fetchall()
    df = pd.DataFrame(rows, columns=['Roll No', 'Name', 'Quiz Total', 'Assignment Total', 'Mid Term', 'Terminal Exam', 'Obtained Marks'])
    print("Marks Fact Table:")
    print(tabulate(df.head(), headers='keys', tablefmt='pretty'))

def main():
    database = "data_warehouse.db"
    dataset_file = "/content/Univeristy_Results.xls"  # Update with your dataset file path

    # Create a database connection
    conn = create_connection(database)
    if conn is not None:
        # Create the Fact table
        create_fact_table(conn)
        # Populate the Fact table
        populate_fact_table(conn, dataset_file)
        # Display the Fact table
        display_fact_table(conn)
        conn.close()
    else:
        print("Error! Cannot create the database connection.")

if __name__ == '__main__':
    main()


Marks Fact Table:
+---+---------+-----------------+------------+------------------+----------+---------------+----------------+
|   | Roll No |      Name       | Quiz Total | Assignment Total | Mid Term | Terminal Exam | Obtained Marks |
+---+---------+-----------------+------------+------------------+----------+---------------+----------------+
| 0 |    1    |  Ashir Mehfooz  |    13.0    |       13.0       |    41    |      41       |     108.0      |
| 1 |    2    |   Atif Raftad   |    4.0     |       4.0        |    30    |      30       |      68.0      |
| 2 |    3    |   Saiqa Aziz    |    14.0    |       14.0       |    34    |      34       |      96.0      |
| 3 |    8    |  Ozair Minhas   |    4.0     |       4.0        |    12    |      12       |      32.0      |
| 4 |    9    | Naveera Subhani |    4.0     |       4.0        |    34    |      34       |      76.0      |
+---+---------+-----------------+------------+------------------+----------+---------------+----------